# Исследование потребительской лояльности 

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России. 

In [1]:
# импорт необходимых библиотек
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

## Получение данных

In [2]:
# Подключение к БД
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

База данных выглядит следующим образом:
![title](https://sun9-13.userapi.com/impg/PZ0Y4b4I366fOmpMzGAQHBqcWl3BLXOoUpfhgw/btiBMCi5-ZU.jpg?size=2560x1175&quality=96&sign=b4be3b3255d68a1ab04b2ae4fb0babbd&type=album)

Итоговый датасет должен содержать:

- user_id	Идентификатор клиента


- lt_day	Количество дней «жизни» клиента


- is_new	Поле хранит информацию о том, является ли клиент новым


- age	Возраст


- gender_segment	Пол (для удобства работы с полем преобразуйте значения в текстовый вид)


- os_name	Тип операционной системы


- cpe_type_name	Тип устройства


- country	Страна проживания


- city	Город проживания


- age_segment	Возрастной сегмент


- traffic_segment	Сегмент по объёму потребляемого трафика


- lifetime_segment	Сегмент по количеству дней «жизни»


- nps_score	Оценка клиента в NPS-опросе


- nps_group	Поле хранит информацию о том, к какой группе относится оценка клиента в опросе

In [3]:
query = """
SELECT user.user_id,
        user.lt_day,
        CASE WHEN user.lt_day < 365 THEN 1 ELSE 0 END as 'is_new',
        user.age,
        CASE WHEN user.gender_segment = 1 THEN 'female'
            WHEN user.gender_segment = 0 THEN 'male'
        END as gender_segment,
        user.os_name,
        user.cpe_type_name,
        location.country,
        location.city,
        SUBSTRING(age_segment.title, 4) as age_segment,
        SUBSTRING(traffic_segment.title, 4) as traffic_segment,
        SUBSTRING(lifetime_segment.title, 4) as lifetime_segment,
        user.nps_score,
        CASE WHEN user.nps_score <= 6 THEN 'detractors' 
            when user.nps_score <= 8 THEN 'passives'
            when user.nps_score <= 10 THEN 'promoters'
        END as nps_group
FROM "user" user
LEFT JOIN location
ON user.location_id = location.location_id 
LEFT JOIN age_segment
ON user.age_gr_id  = age_segment.age_gr_id  
LEFT JOIN traffic_segment 
ON user.tr_gr_id = traffic_segment.tr_gr_id   
LEFT JOIN lifetime_segment 
ON user.lt_gr_id = lifetime_segment.lt_gr_id   
"""

In [4]:
df = pd.read_sql(query, engine)

In [5]:
df.head()

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,0,45.0,female,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,promoters
1,A001WF,2344,0,53.0,male,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,promoters
2,A003Q7,467,0,57.0,male,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,promoters
3,A004TB,4190,0,44.0,female,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,promoters
4,A004XT,1163,0,24.0,male,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,promoters


In [6]:
df.shape

(502493, 14)

In [7]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

## Дашборды и story

https://public.tableau.com/app/profile/alexandr.baranov/viz/NPS_16846118840790/Story1